<a href="https://colab.research.google.com/github/peremartra/LLMOptCost/blob/main/11/12_pruning_structured.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn.utils.prune as prune
from transformers import AutoModelForCausalLM, AutoTokenizer
import matplotlib.pyplot as plt

In [ ]:
def check_sparsity(layer):
    zero_weights = torch.sum(layer.weight == 0).item()
    total_weights = layer.weight.numel()
    sparsity = zero_weights / total_weights
    print(f"Sparsity: {sparsity:.2%} of the weights are zero.")

In [ ]:
# Load  Hugging Face model
base_model_name = "bigscience/bloomz-560m"
pruned_model_name = "bloomz-560m-pruned-structured"
model = AutoModelForCausalLM.from_pretrained(base_model_name)

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained('bigscience/bloomz-560m')

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
def compute_neuron_importance(mlp_layer):
    # Access the first linear layer in the MLP
    dense_h_to_4h = mlp_layer.dense_h_to_4h
    weight = dense_h_to_4h.weight.data  # Shape: (intermediate_size, hidden_size)

    # Compute the L2 norm of each neuron (over input features)
    neuron_importance = torch.norm(weight, p=2, dim=1)  # Shape: (intermediate_size,)

    return neuron_importance

In [ ]:
def select_neurons_to_prune(neuron_importance, pruning_percentage):
    num_neurons = neuron_importance.size(0)
    num_prune = int(num_neurons * pruning_percentage)

    # Get indices of neurons sorted by importance
    sorted_indices = torch.argsort(neuron_importance)

    # Select indices to prune (least important neurons)
    neurons_to_prune = sorted_indices[:num_prune]

    return neurons_to_prune

In [ ]:
def prune_mlp_neurons(mlp_layer, neurons_to_prune):
    """
    Prune specified neurons in the MLP layer.
    """
    # Access the MLP's linear layers
    dense_h_to_4h = mlp_layer.dense_h_to_4h
    dense_4h_to_h = mlp_layer.dense_4h_to_h

    # Prune neurons in dense_h_to_4h (input to MLP)
    prune_linear_layer(dense_h_to_4h, neurons_to_prune, dim=0)

    # Prune corresponding neurons in dense_4h_to_h (output of MLP)
    prune_linear_layer(dense_4h_to_h, neurons_to_prune, dim=1)

In [ ]:
def prune_linear_layer(layer, indices_to_prune, dim=0):
    """
    Prune specified indices from a linear layer along the given dimension.
    """
    # Convert indices to keep
    indices_to_keep = torch.tensor([
        idx for idx in range(layer.weight.size(dim))
        if idx not in set(indices_to_prune.tolist())
    ])

    # Prune weights
    weight = layer.weight.data.index_select(dim, indices_to_keep)
    layer.weight = torch.nn.Parameter(weight)

    # Prune biases if necessary
    if layer.bias is not None:
        if dim == 0:
            bias = layer.bias.data.index_select(0, indices_to_keep)
            layer.bias = torch.nn.Parameter(bias)

    # Update layer dimensions
    if dim == 0:
        layer.out_features = weight.size(0)
    else:
        layer.in_features = weight.size(1)

In [ ]:
def prune_mlp_neurons_in_model(model, pruning_percentage):
    num_layers = model.config.num_hidden_layers

    for layer_idx in range(num_layers):
        # Access the MLP layer
        mlp_layer = model.transformer.h[layer_idx].mlp

        # Compute neuron importance
        neuron_importance = compute_neuron_importance(mlp_layer)

        # Select neurons to prune
        neurons_to_prune = select_neurons_to_prune(neuron_importance, pruning_percentage)

        # Prune neurons
        prune_mlp_neurons(mlp_layer, neurons_to_prune)

        # Update intermediate_size in the configuration
        new_intermediate_size = mlp_layer.dense_h_to_4h.out_features
        model.config.hidden_dropout = new_intermediate_size  # Update if necessary


In [ ]:
# Before pruning
total_params_before = count_parameters(model)
print(f"Total parameters before pruning: {total_params_before}")

Total parameters before pruning: 559214592


In [ ]:
prompt="Tell me a joke"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
base_output = model.generate(**inputs)
base_response = tokenizer.decode(base_output[0], skip_special_tokens=True)
print(base_response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Tell me a joke about a man who is a doctor


In [ ]:
# Define the pruning percentage (e.g., 20% pruning)
pruning_percentage = 0.3  # Adjust as needed

# Prune MLP neurons in the model
prune_mlp_neurons_in_model(model, pruning_percentage)

In [ ]:
# After pruning
total_params_after = count_parameters(model)
print(f"Total parameters after pruning: {total_params_after}")

# Calculate the number of parameters removed
params_removed = total_params_before - total_params_after
print(f"Number of parameters removed: {params_removed}")

# Calculate percentage reduction
percent_reduction = 100.0 * params_removed / total_params_before
print(f"Percentage reduction in parameters: {percent_reduction:.2f}%")


Total parameters after pruning: 498826464
Number of parameters removed: 60388128
Percentage reduction in parameters: 10.80%


In [ ]:
prompt="Tell me a joke"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
base_output = model.generate(**inputs)
base_response = tokenizer.decode(base_output[0], skip_special_tokens=True)
print(base_response)

Tell me a joke about how how you got to to the first first first time you can get to


# Save & Upload Model

In [ ]:
# Save the quantized model
model.save_pretrained(
    pruned_model_name,
)
tokenizer.save_pretrained(pruned_model_name)

('bloomz-560m-pruned-structured/tokenizer_config.json',
 'bloomz-560m-pruned-structured/special_tokens_map.json',
 'bloomz-560m-pruned-structured/tokenizer.json')

In [ ]:
model.push_to_hub(pruned_model_name,
                  private=False,
                  use_temp_dir=False)

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/oopere/bloomz-560m-pruned-structured/commit/b4fdc5b12d458484be8090b4dda73029892b682e', commit_message='Upload BloomForCausalLM', commit_description='', oid='b4fdc5b12d458484be8090b4dda73029892b682e', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub(pruned_model_name,
                      private=False,
                      use_temp_dir=False)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/oopere/bloomz-560m-pruned-structured/commit/f292dfd57da7954faea6ef8276b7c23a9e8cbf8f', commit_message='Upload tokenizer', commit_description='', oid='f292dfd57da7954faea6ef8276b7c23a9e8cbf8f', pr_url=None, pr_revision=None, pr_num=None)